# FINAL PROJECT

In [ ]:
import sys
import subprocess
import warnings
warnings.filterwarnings('ignore')

def install_packages():
    packages = [
        'scikit-learn', 'pandas', 'numpy', 'matplotlib', 
        'seaborn', 'ipywidgets', 'joblib', 'imbalanced-learn',
        'jinja2>=3.0.0'
    ]
    print("📦 Installing/verifying required packages...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade"] + packages)
    print("✅ All packages ready!\n")

install_packages()

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, f1_score, accuracy_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output
import joblib

sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

print("=" * 70)
print("🚀 TOXIC COMMENT CLASSIFIER - STARTING...")
print("=" * 70)

def load_dataset():
    print("\n📥 Loading final enlarged dataset...")

    try:
        path = "expanded_toxic_comment_dataset.csv"
        df = pd.read_csv(path)
        print(f"✅ Loaded {len(df)} samples from {path}")

        # ✅ Ensure correct structure
        if 'comment_text' not in df.columns:
            text_col = df.columns[0]
            df.rename(columns={text_col: 'comment_text'}, inplace=True)

        if 'toxic' not in df.columns:
            raise ValueError("Dataset must include 'toxic' column with 0/1 labels")

        # ✅ Basic cleaning
        df['comment_text'] = df['comment_text'].astype(str).str.strip()
        df.dropna(subset=['comment_text'], inplace=True)
        df.drop_duplicates(subset=['comment_text'], inplace=True)
        df = df[df['comment_text'].str.len() > 3].reset_index(drop=True)

        # ✅ Display stats
        print(f"📊 Final dataset size: {len(df)}")
        print(f"🧩 Toxic samples: {df['toxic'].sum()} | Non-Toxic samples: {len(df) - df['toxic'].sum()}")

    except Exception as e:
        print(f"⚠️ Error loading dataset: {e}")
        print("📝 Creating small demo dataset as fallback...")
        data = []
        for i in range(2000):
            if i % 2 == 0:
                data.append({'comment_text': 'You are stupid and worthless', 'toxic': 1})
            else:
                data.append({'comment_text': 'Thank you for your help', 'toxic': 0})
        df = pd.DataFrame(data)
        print("✅ Demo dataset created successfully!")

    return df

df = load_dataset()

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#\w+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("🧹 Preprocessing data...")
df['cleaned_text'] = df['comment_text'].apply(clean_text)
df = df[df['cleaned_text'].str.len() > 0].reset_index(drop=True)

X = df['cleaned_text']
y = df['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("🔤 Extracting features...")
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 3), min_df=2, max_df=0.95, 
                        sublinear_tf=True, stop_words='english')
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

class_ratio = y_train.value_counts()[0] / y_train.value_counts()[1]
if class_ratio > 1.5 or class_ratio < 0.67:
    print("⚖️ Balancing classes with SMOTE...")
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train_tfidf, y_train)
else:
    X_train_balanced, y_train_balanced = X_train_tfidf, y_train

print("🤖 Training models...")
lr_model = LogisticRegression(max_iter=1000, solver='saga', C=1.0, 
                              class_weight='balanced', random_state=42, n_jobs=-1)
lr_model.fit(X_train_balanced, y_train_balanced)

nb_model = MultinomialNB(alpha=0.1)
nb_model.fit(X_train_balanced, y_train_balanced)

rf_model = RandomForestClassifier(n_estimators=100, max_depth=50, 
                                  class_weight='balanced', random_state=42, n_jobs=-1)
rf_model.fit(X_train_balanced, y_train_balanced)

ensemble_model = VotingClassifier(
    estimators=[('lr', lr_model), ('nb', nb_model), ('rf', rf_model)],
    voting='soft', n_jobs=-1
)
ensemble_model.fit(X_train_balanced, y_train_balanced)

models = {
    'Logistic Regression': lr_model,
    'Naive Bayes': nb_model,
    'Random Forest': rf_model,
    'Ensemble': ensemble_model
}

print("📊 Evaluating models...")
results = []
for name, model in models.items():
    y_pred = model.predict(X_test_tfidf)
    y_proba = model.predict_proba(X_test_tfidf)[:, 1]
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_proba)
    })

best_model = lr_model
model_name = "Logistic Regression"
y_pred = best_model.predict(X_test_tfidf)
y_proba = best_model.predict_proba(X_test_tfidf)[:, 1]
cm = confusion_matrix(y_test, y_pred)
feature_names = np.array(tfidf.get_feature_names_out())

print("💾 Saving models...")
joblib.dump(best_model, 'toxic_classifier_model.joblib')
joblib.dump(tfidf, 'tfidf_vectorizer.joblib')
joblib.dump(ensemble_model, 'toxic_classifier_ensemble.joblib')

print("\n" + "=" * 70)
print("✅ MODEL TRAINING COMPLETE!")
print("=" * 70 + "\n")

display(HTML("""
<style>
.dashboard-container {
    max-width: 1400px;
    margin: 0 auto;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}
.section-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 20px;
    border-radius: 10px;
    margin: 20px 0;
    text-align: center;
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);
}
.metrics-grid {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
    gap: 15px;
    margin: 20px 0;
}
.metric-card {
    background: white;
    border-left: 4px solid #667eea;
    padding: 20px;
    border-radius: 8px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.metric-value {
    font-size: 32px;
    font-weight: bold;
    color: #667eea;
}
.metric-label {
    font-size: 14px;
    color: #666;
    margin-top: 5px;
}
.prediction-section {
    background: #f8f9fa;
    border-radius: 10px;
    padding: 25px;
    margin: 20px 0;
}
</style>
<div class="dashboard-container">
    <div class="section-header">
        <h1>🎯 TOXIC COMMENT CLASSIFIER DASHBOARD</h1>
        <p>Advanced ML-Powered Content Moderation System</p>
    </div>
</div>
"""))

display(Markdown("---"))
display(Markdown("# 📊 SECTION 1: MODEL PERFORMANCE & ANALYSIS"))

results_df = pd.DataFrame(results)

display(Markdown("### 🏆 Model Performance Comparison"))
try:
    display(results_df.style.highlight_max(axis=0, subset=['Accuracy', 'F1-Score', 'ROC-AUC']))
except:
    display(results_df)

fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

ax1 = fig.add_subplot(gs[0, :])
results_df.plot(x='Model', kind='bar', ax=ax1, rot=0, colormap='viridis')
ax1.set_title('📊 Model Performance Comparison', fontsize=16, fontweight='bold')
ax1.set_ylabel('Score')
ax1.set_ylim([0, 1])
ax1.legend(loc='lower right')
ax1.grid(alpha=0.3)

ax2 = fig.add_subplot(gs[1, 0])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Toxic', 'Toxic'],
            yticklabels=['Non-Toxic', 'Toxic'], ax=ax2, cbar=False)
ax2.set_title('Confusion Matrix', fontweight='bold')

ax3 = fig.add_subplot(gs[1, 1])
fpr, tpr, _ = roc_curve(y_test, y_proba)
auc_score = roc_auc_score(y_test, y_proba)
ax3.plot(fpr, tpr, linewidth=2, label=f'AUC = {auc_score:.3f}', color='#667eea')
ax3.plot([0, 1], [0, 1], 'k--', linewidth=1)
ax3.set_xlabel('False Positive Rate')
ax3.set_ylabel('True Positive Rate')
ax3.set_title('ROC Curve', fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

ax4 = fig.add_subplot(gs[1, 2])
coefficients = best_model.coef_[0]
top_toxic_idx = np.argsort(coefficients)[-10:]
top_toxic_features = feature_names[top_toxic_idx]
top_toxic_coefs = coefficients[top_toxic_idx]
ax4.barh(range(len(top_toxic_features)), top_toxic_coefs, color='red', alpha=0.7)
ax4.set_yticks(range(len(top_toxic_features)))
ax4.set_yticklabels(top_toxic_features, fontsize=9)
ax4.set_xlabel('Coefficient')
ax4.set_title('Top 10 Toxic Indicators', fontweight='bold', color='darkred')
ax4.grid(axis='x', alpha=0.3)

ax5 = fig.add_subplot(gs[2, :2])
top_nontoxic_idx = np.argsort(coefficients)[:15]
top_nontoxic_features = feature_names[top_nontoxic_idx]
top_nontoxic_coefs = coefficients[top_nontoxic_idx]
ax5.barh(range(len(top_nontoxic_features)), top_nontoxic_coefs, color='green', alpha=0.7)
ax5.set_yticks(range(len(top_nontoxic_features)))
ax5.set_yticklabels(top_nontoxic_features, fontsize=9)
ax5.set_xlabel('Coefficient')
ax5.set_title('Top 15 Non-Toxic Indicators', fontweight='bold', color='darkgreen')
ax5.grid(axis='x', alpha=0.3)

ax6 = fig.add_subplot(gs[2, 2])
metrics_data = {
    'Metric': ['Accuracy', 'F1-Score', 'Precision', 'Recall'],
    'Score': [
        accuracy_score(y_test, y_pred),
        f1_score(y_test, y_pred),
        cm[1,1]/(cm[1,1]+cm[0,1]) if (cm[1,1]+cm[0,1]) > 0 else 0,
        cm[1,1]/(cm[1,1]+cm[1,0]) if (cm[1,1]+cm[1,0]) > 0 else 0
    ]
}
bars = ax6.bar(metrics_data['Metric'], metrics_data['Score'], color=['#667eea', '#764ba2', '#f093fb', '#4facfe'])
ax6.set_ylim([0, 1])
ax6.set_title('Key Metrics Summary', fontweight='bold')
ax6.set_ylabel('Score')
for bar in bars:
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
ax6.grid(axis='y', alpha=0.3)

plt.suptitle('🎯 Comprehensive Model Analysis Dashboard', fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

accuracy_val = accuracy_score(y_test, y_pred)
f1_val = f1_score(y_test, y_pred)
auc_val = roc_auc_score(y_test, y_proba)

summary_html = f"""
<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            border-radius: 15px; padding: 30px; color: white; margin: 20px 0;
            box-shadow: 0 8px 16px rgba(0,0,0,0.2);'>
    <h2 style='margin-top: 0; text-align: center; border-bottom: 2px solid white; padding-bottom: 15px;'>
        📊 Model Performance Summary
    </h2>
    <div class="metrics-grid" style='display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 20px; margin-top: 20px;'>
        <div style='background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; backdrop-filter: blur(10px);'>
            <div style='font-size: 36px; font-weight: bold;'>{accuracy_val*100:.1f}%</div>
            <div style='font-size: 14px; opacity: 0.9;'>Accuracy</div>
        </div>
        <div style='background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; backdrop-filter: blur(10px);'>
            <div style='font-size: 36px; font-weight: bold;'>{f1_val:.3f}</div>
            <div style='font-size: 14px; opacity: 0.9;'>F1-Score</div>
        </div>
        <div style='background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; backdrop-filter: blur(10px);'>
            <div style='font-size: 36px; font-weight: bold;'>{auc_val:.3f}</div>
            <div style='font-size: 14px; opacity: 0.9;'>ROC-AUC</div>
        </div>
        <div style='background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; backdrop-filter: blur(10px);'>
            <div style='font-size: 36px; font-weight: bold;'>{len(df):,}</div>
            <div style='font-size: 14px; opacity: 0.9;'>Total Samples</div>
        </div>
    </div>
    <div style='margin-top: 25px; padding: 20px; background: rgba(255,255,255,0.1); border-radius: 10px; backdrop-filter: blur(10px);'>
        <strong>📁 Saved Models:</strong><br>
        ✅ toxic_classifier_model.joblib<br>
        ✅ tfidf_vectorizer.joblib<br>
        ✅ toxic_classifier_ensemble.joblib
    </div>
</div>
"""
display(HTML(summary_html))

display(Markdown("---"))
display(Markdown("# 🎯 SECTION 2: LIVE COMMENT ANALYSIS"))

display(HTML("""
<div style='background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
            border-radius: 15px; padding: 30px; color: white; text-align: center;
            box-shadow: 0 8px 16px rgba(0,0,0,0.2); margin: 20px 0;'>
    <h2 style='margin: 0;'>🚀 Real-Time Toxicity Analyzer</h2>
    <p style='margin: 10px 0 0 0; opacity: 0.9;'>Enter any comment below for instant analysis</p>
</div>
"""))

style = {'description_width': '130px'}

comment_input = widgets.Textarea(
    value='',
    placeholder='Type or paste your comment here... (e.g., "Thank you for helping!" or "You are stupid")',
    description='Your Comment:',
    layout=widgets.Layout(width='98%', height='140px'),
    style=style
)

model_selector = widgets.Dropdown(
    options=['Logistic Regression', 'Naive Bayes', 'Random Forest', 'Ensemble'],
    value='Ensemble',
    description='Select Model:',
    style=style
)

threshold_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Threshold:',
    style=style,
    readout_format='.2f'
)

analyze_btn = widgets.Button(
    description='🔍 ANALYZE TOXICITY',
    button_style='danger',
    layout=widgets.Layout(width='280px', height='50px'),
    style={'font_weight': 'bold'}
)

clear_btn = widgets.Button(
    description='🗑️ Clear',
    button_style='warning',
    layout=widgets.Layout(width='150px', height='50px')
)

output_area = widgets.Output()

def analyze_comment(b):
    with output_area:
        clear_output()
        
        text = comment_input.value.strip()
        if not text:
            display(HTML("<div style='padding: 20px; background: #fff3cd; border-left: 5px solid #ffc107; border-radius: 5px;'><strong>⚠️ Please enter a comment to analyze!</strong></div>"))
            return
        
        cleaned = clean_text(text)
        if not cleaned:
            display(HTML("<div style='padding: 20px; background: #fff3cd; border-left: 5px solid #ffc107; border-radius: 5px;'><strong>⚠️ Text is empty after cleaning!</strong></div>"))
            return
        
        selected_model = models[model_selector.value]
        text_tfidf = tfidf.transform([cleaned])
        proba = selected_model.predict_proba(text_tfidf)[0, 1]
        threshold = threshold_slider.value
        prediction = 'TOXIC' if proba >= threshold else 'NON-TOXIC'
        
        color = '#d32f2f' if prediction == 'TOXIC' else '#388e3c'
        bg_gradient = 'linear-gradient(135deg, #ffebee 0%, #ffcdd2 100%)' if prediction == 'TOXIC' else 'linear-gradient(135deg, #e8f5e9 0%, #c8e6c9 100%)'
        emoji = '🚫' if prediction == 'TOXIC' else '✅'
        icon = '⚠️' if prediction == 'TOXIC' else '👍'
        
        display(HTML(f"""
        <div style='background: {bg_gradient}; border: 4px solid {color}; 
                    border-radius: 20px; padding: 35px; margin: 20px 0;
                    box-shadow: 0 10px 25px rgba(0,0,0,0.15);'>
            <div style='text-align: center;'>
                <h1 style='color: {color}; margin: 0; font-size: 48px;'>
                    {emoji} {prediction}
                </h1>
                <div style='margin: 25px 0; padding: 25px; background: white; 
                            border-radius: 15px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);'>
                    <div style='font-size: 18px; color: #666; margin-bottom: 10px;'>
                        <strong>Toxicity Score</strong>
                    </div>
                    <div style='font-size: 56px; font-weight: bold; color: {color};'>
                        {proba*100:.1f}%
                    </div>
                    <div style='margin-top: 15px; font-size: 14px; color: #999;'>
                        Model: {model_selector.value} | Threshold: {threshold:.2f}
                    </div>
                </div>
            </div>
            
            <div style='background: rgba(255,255,255,0.8); padding: 20px; 
                        border-radius: 10px; margin-top: 20px; border-left: 5px solid {color};'>
                <strong style='color: #333;'>📝 Your Comment:</strong><br>
                <em style='color: #666; font-size: 16px; line-height: 1.6;'>
                    "{text[:300]}{'...' if len(text) > 300 else ''}"
                </em>
            </div>
            
            <div style='background: {"#fff3e0" if prediction == "TOXIC" else "#e3f2fd"}; 
                        padding: 20px; border-radius: 10px; margin-top: 20px;
                        border-left: 5px solid {"#ff9800" if prediction == "TOXIC" else "#2196f3"};'>
                <strong>{icon} {"Warning" if prediction == "TOXIC" else "Great"}:</strong>
                {"This comment may violate community guidelines. Consider rephrasing to be more respectful." 
                 if prediction == "TOXIC" 
                 else "This comment appears to be respectful and constructive. Keep up the positive communication!"}
            </div>
        </div>
        """))
        
        if isinstance(selected_model, LogisticRegression):
            vec = text_tfidf.toarray()[0]
            contributions = vec * selected_model.coef_[0]
            nonzero_idx = np.where(vec > 0)[0]
            
            if len(nonzero_idx) > 0:
                contrib_list = [(feature_names[idx], contributions[idx]) for idx in nonzero_idx]
                contrib_sorted = sorted(contrib_list, key=lambda x: abs(x[1]), reverse=True)[:8]
                
                display(HTML("<div style='margin-top: 25px;'><h3 style='color: #333;'>🔍 Key Words Detected:</h3></div>"))
                
                words_html = "<div style='display: flex; flex-wrap: wrap; gap: 10px; margin-top: 15px;'>"
                for word, score in contrib_sorted:
                    if score > 0:
                        words_html += f"""
                        <div style='padding: 12px 20px; background: #ffcdd2; border-radius: 25px; 
                                    border: 2px solid #e57373; display: inline-block;'>
                            <strong style='color: #c62828;'>🔴 {word}</strong> 
                            <span style='color: #666; font-size: 11px;'>(+{score:.3f})</span>
                        </div>
                        """
                    else:
                        words_html += f"""
                        <div style='padding: 12px 20px; background: #c8e6c9; border-radius: 25px; 
                                    border: 2px solid #81c784; display: inline-block;'>
                            <strong style='color: #2e7d32;'>🟢 {word}</strong> 
                            <span style='color: #666; font-size: 11px;'>({score:.3f})</span>
                        </div>
                        """
                words_html += "</div>"
                display(HTML(words_html))

def clear_input(b):
    comment_input.value = ''
    with output_area:
        clear_output()

analyze_btn.on_click(analyze_comment)
clear_btn.on_click(clear_input)

controls = widgets.VBox([
    comment_input,
    widgets.HBox([model_selector, threshold_slider], layout=widgets.Layout(justify_content='space-between')),
    widgets.HBox([analyze_btn, clear_btn], layout=widgets.Layout(gap='15px')),
], layout=widgets.Layout(padding='15px'))

display(controls)
display(output_area)

display(HTML("""
<div style='background: #f8f9fa; border-radius: 10px; padding: 25px; margin: 30px 0;'>
    <h3 style='color: #333; margin-top: 0;'>💡 Try These Examples:</h3>
    <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 15px;'>
        <div style='padding: 15px; background: #c8e6c9; border-radius: 8px; border-left: 4px solid #4caf50;'>
            <strong>✅ Non-Toxic:</strong><br>
            <em>"Thank you for your help! This is great."</em>
        </div>
        <div style='padding: 15px; background: #c8e6c9; border-radius: 8px; border-left: 4px solid #4caf50;'>
            <strong>✅ Non-Toxic:</strong><br>
            <em>"I respectfully disagree with your opinion."</em>
        </div>
        <div style='padding: 15px; background: #ffcdd2; border-radius: 8px; border-left: 4px solid #f44336;'>
            <strong>🚫 Toxic:</strong><br>
            <em>"You are stupid and worthless."</em>
        </div>
        <div style='padding: 15px; background: #ffcdd2; border-radius: 8px; border-left: 4px solid #f44336;'>
            <strong>🚫 Toxic:</strong><br>
            <em>"Shut up idiot, nobody cares."</em>
        </div>
    </div>
</div>
"""))
display(Markdown("## 📋 Final Summary Report"))

try:
    # Determine the best model based on F1-Score (you can change to 'Accuracy' if preferred)
    best_row = results_df.loc[results_df['F1-Score'].idxmax()]
    model_name = best_row['Model']
    best_model = models[model_name]

    # Evaluate best model again to get metrics
    y_pred = best_model.predict(X_test_tfidf)
    y_proba = best_model.predict_proba(X_test_tfidf)[:, 1]
    accuracy_val = accuracy_score(y_test, y_pred)
    f1_val = f1_score(y_test, y_pred)
    auc_val = roc_auc_score(y_test, y_proba)
    precision_val = cm[1,1] / (cm[1,1] + cm[0,1]) if (cm[1,1]+cm[0,1]) > 0 else 0
    recall_val = cm[1,1] / (cm[1,1] + cm[1,0]) if (cm[1,1]+cm[1,0]) > 0 else 0
    total_pos = int(sum(y_test))
    total_neg = int(len(y_test) - total_pos)

    summary_html = f"""
    <div style='border-radius: 12px; background: linear-gradient(135deg, #eef2ff 0%, #ffffff 100%);
                padding: 25px 30px; box-shadow: 0 6px 12px rgba(0,0,0,0.1);'>
        <h2 style='color: #333; text-align:center; margin-top:0;'>
            🎯 <span style="color:#4a60e8;">Toxic Comment Classifier</span> - Final Performance Summary
        </h2>
        <hr style="border: none; border-top: 2px solid #4a60e8; margin: 15px 0 25px 0;">

        <div style='display: flex; flex-wrap: wrap; justify-content: space-around; gap: 20px;'>
            <div style='background:#f9faff; border-left:5px solid #4a60e8; border-radius:10px;
                        padding:15px 20px; flex:1; min-width:220px;'>
                <h4 style='margin:5px 0;'>📊 Dataset Info</h4>
                <ul style='line-height:1.7;'>
                    <li><b>Total Samples:</b> {len(df):,}</li>
                    <li><b>Training Samples:</b> {len(X_train):,}</li>
                    <li><b>Test Samples:</b> {len(X_test):,}</li>
                    <li><b>Features (TF-IDF):</b> {X_train_tfidf.shape[1]:,}</li>
                    <li><b>Non-Toxic:</b> {total_neg:,} | <b>Toxic:</b> {total_pos:,}</li>
                </ul>
            </div>

            <div style='background:#f9faff; border-left:5px solid #4a60e8; border-radius:10px;
                        padding:15px 20px; flex:1; min-width:220px;'>
                <h4 style='margin:5px 0;'>🏆 Best Model Performance ({model_name})</h4>
                <ul style='line-height:1.7;'>
                    <li><b>Accuracy:</b> {accuracy_val:.4f} ({accuracy_val*100:.2f}%)</li>
                    <li><b>F1-Score:</b> {f1_val:.4f}</li>
                    <li><b>Precision:</b> {precision_val:.4f}</li>
                    <li><b>Recall:</b> {recall_val:.4f}</li>
                    <li><b>ROC-AUC:</b> {auc_val:.4f}</li>
                </ul>
            </div>

            <div style='background:#f9faff; border-left:5px solid #4a60e8; border-radius:10px;
                        padding:15px 20px; flex:1; min-width:220px;'>
                <h4 style='margin:5px 0;'>📈 Confusion Matrix</h4>
                <ul style='line-height:1.7;'>
                    <li><b>True Positives:</b> {cm[1,1]}</li>
                    <li><b>True Negatives:</b> {cm[0,0]}</li>
                    <li><b>False Positives:</b> {cm[0,1]}</li>
                    <li><b>False Negatives:</b> {cm[1,0]}</li>
                    <li><b>Threshold:</b> {threshold_slider.value:.2f}</li>
                </ul>
            </div>
        </div>

        <div style='margin-top:30px; background:#eef3ff; border-radius:10px;
                    padding:15px 20px;'>
            <h4>💾 Saved Model Artifacts</h4>
            <ul style='line-height:1.7;'>
                <li>✅ toxic_classifier_model.joblib</li>
                <li>✅ tfidf_vectorizer.joblib</li>
                <li>✅ toxic_classifier_ensemble.joblib</li>
            </ul>
        </div>

        <div style='margin-top:25px; background:#f9fcff; border-radius:10px;
                    padding:15px 20px; border-left:4px solid #4a60e8;'>
            <h4>🚀 Next Steps</h4>
            <ol style='line-height:1.8;'>
                <li>Use the interactive panel above to test live comments.</li>
                <li>Experiment with the threshold slider to balance precision and recall.</li>
                <li>Try other models (Naive Bayes, Random Forest, Ensemble) from the dropdown.</li>
                <li>Load your model anytime:
                    <code>model = joblib.load('toxic_classifier_model.joblib')</code>
                </li>
                <li>For production, fine-tune on a larger dataset such as Jigsaw (160K+ rows).</li>
                <li>Optionally explore deep learning (BERT, RoBERTa) for even higher accuracy.</li>
            </ol>
        </div>

        <div style='margin-top:25px; padding:15px; background:#e8f4ff; border-left:4px solid #2196f3;
                    border-radius:8px;'>
            <strong>💡 Pro Tip:</strong> Continue expanding your dataset with
            context-rich examples (neutral criticism, greetings, sarcasm) to reduce false positives and negatives.
        </div>
    </div>
    """

    display(HTML(summary_html))

except Exception as e:
    print(f"⚠️ Could not generate summary report: {e}")
    print("Model training completed successfully. Check outputs above for results.")
